In [1]:
#1
#시 token data 불러오기
import pandas as pd
import numpy as np

poems_token = pd.read_csv("C:\DeepLearing\Robot_KSG\poem\poems_token_mecab.csv", index_col=0) #index_col=0: 맨앞의 열 인덱스로 사용(인덱스 재부여 방지)
poems_token = poems_token['본문'] #본문 column만 사용

poems_token

0       ['1', '2', '3', '4', '선생', '서덕출', '학교', '조그마',...
1       ['10', '월', '노래', '짓밟힌', '무리', '흘린', '핏방울', '방...
2       ['일간', '15', '깨끗', '시트', '위', '몸부림', '쳐도', '소용...
3       ['김창', '술독', '와사', '퍼트린', '드', '시', '숨', '맥히',...
4       ['김창', '술', '울', '엉', '찬', '새벽', '고동', '잠들', '...
                              ...                        
4869    ['확실히', '소리', '소리', '들리', '안', '씹는', '수많', '깨'...
4870    ['소리', '김대봉', '찬', '서리', '나리', '고요', '밤', '낙엽'...
4871    ['논개', '거룩', '분노', '종교', '깊', '불붙', '정열', '사랑'...
4872    ['육', '화육', '릉', '향락', '단명', '걸', '알', '물', '위...
4873    ['가등', '졸음', '못', '밤거리', '이성', '휑', '비인', '거리'...
Name: 본문, Length: 4874, dtype: object

In [2]:
#2.0
#1차 정수 인코딩
from gensim import corpora
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(poems_token) #정수 인코딩(tokenizer에 저장, 빈도수 높은 순)

print(tokenizer.word_index)

{"'내'": 1, "'없'": 2, "'보'": 3, "'일'": 4, "'기'": 5, "'말'": 6, "'사람'": 7, "'속'": 8, "'않'": 9, "'눈'": 10, "'마음'": 11, "'시'": 12, "'소리'": 13, "'안'": 14, "'밤'": 15, "'길'": 16, "'수'": 17, "'날'": 18, "'듯'": 19, "'할'": 20, "'해'": 21, "'사랑'": 22, "'두'": 23, "'년'": 24, "'가슴'": 25, "'님'": 26, "'주'": 27, "'생각'": 28, "'처럼'": 29, "'하늘'": 30, "'바람'": 31, "'못'": 32, "'물'": 33, "'다시'": 34, "'피'": 35, "'노래'": 36, "'몸'": 37, "'우'": 38, "'눈물'": 39, "'달'": 40, "'알'": 41, "'위'": 42, "'손'": 43, "'집'": 44, "'줄'": 45, "'새'": 46, "'울'": 47, "'살'": 48, "'당신'": 49, "'그대'": 50, "'불'": 51, "'적'": 52, "'봄'": 53, "'바다'": 54, "'곳'": 55, "'죽'": 56, "'영철'": 57, "'꽃'": 58, "'없이'": 59, "'업'": 60, "'산'": 61, "'얼굴'": 62, "'남'": 63, "'그러'": 64, "'치'": 65, "'설화'": 66, "'못하'": 67, "'가지'": 68, "'모든'": 69, "'오늘'": 70, "'어서'": 71, "'꿈'": 72, "'사'": 73, "'월'": 74, "'번'": 75, "'십'": 76, "'비'": 77, "'삼'": 78, "'별'": 79, "'히'": 80, "'모르'": 81, "'웃'": 82, "'앞'": 83, "'타'": 84, "'구'": 85, "'땅'": 86, "'그것'": 87, "'나라'": 88, "'어머니'": 89,

In [4]:
#2.0
#단어 비율 확인

threshold = 3 #threshold: 임계값
total_cnt = len(tokenizer.word_index) #total_cnt: 전체 단어 수
rare_cnt = 0 #rare_cnt: 등장 빈도수 0~2인 단어 count
total_freq = 0 #total_freq: 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 #rare_freq: 등장 빈도수 0~2인 단어의 빈도수 총 합

for key, value in tokenizer.word_counts.items(): #key, value: 단어, 빈도수
    total_freq += value
    
    if(value < threshold):
        rare_cnt += 1
        rare_freq += value

print('vocabulary 크기 : ', total_cnt)
print('등장 빈도 %s 이하인 희귀 단어 수 : %s' %(threshold-1, rare_cnt))
print("희귀 단어 비율 : ", (rare_cnt/total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : ", (rare_freq/total_freq)*100)

vocabulary 크기 :  41538
등장 빈도 2 이하인 희귀 단어 수 : 25711
희귀 단어 비율 :  61.89753960229187
전체 등장 빈도에서 희귀 단어 등장 빈도 비율 :  7.177994780368311


In [5]:
#2
#빈도수 3 미만 단어 제거하며 정수 인코딩 + 정수-단어 사전 생성

vocab_size = total_cnt - rare_cnt + 2 #+2: 0번- 패딩토큰, 1번- OOV토큰

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(poems_token)

dictionary = tokenizer.word_index #dictionary: word_index 딕셔너리 변수로 저장
dictionary = {v:k for k, v in dictionary.items() if v<=vocab_size} #key, value: 정수, 단어 로 수정(vocab_size만큼만 가져와서 뒤집기)

poems_token_int = tokenizer.texts_to_sequences(poems_token) #poems_token_int: 정수 단어 토큰

print(len(dictionary))
poems_token_int[101]

15829


[4701,
 12,
 1478,
 1478,
 2814,
 323,
 1,
 6,
 189,
 1,
 12681,
 52,
 1219,
 5,
 3929,
 12682,
 5018,
 2897,
 52,
 4393,
 5,
 3929,
 3930,
 5018,
 4394,
 65,
 32,
 52,
 1,
 60,
 235,
 12528,
 330]

In [116]:
#빈도수 낮은 데이터로만 구성된 샘플 있나 확인
drop_train = [index for index, sentence in enumerate(poems_token_int) if len(sentence)<1]

print(drop_train)

[]


In [117]:
dictionary[vocab_size]

"'굴린다'"